**FlashAttention**

FlashAttention 是一套精确（exact）的注意力加速算法（内核），核心思想不是改数学公式、也不是近似，而是让注意力计算更贴近 GPU 的存储层级与并行特性：把 Q / K / V 分块（tiling）放进片上 SRAM，采用在线 softmax 与重算策略，最大化实现所谓算在片上、少碰显存，从而显著减少 HBM 读写、降低内存占用并提升速度（训练与推理都受益）。

<div align="center">
<img src="https://github.com/Dao-AILab/flash-attention/blob/main/assets/flashattn_banner.jpg?raw=true" alt="fused kernel" width="40%">
</div>


三代演进。
- FA1（2022）：提出 IO-aware 精确注意力；通过分块 + 在线 softmax / 重算，显著减少显存访问与内存峰值，带来端到端训练加速（例如 GPT-2 / 长序列基准上多倍加速）；
- FA2（2023）：主要做并行与工作划分的工程优化——提高线程块 / warp 的占用率，减少非 matmul 的开销，让内核效率更接近 GEMM；论文报告相比 FA1 约 ~2× 速度提升，在 A100 上达到 50–73% 理论 FLOPs 利用率（端到端训练也验证了加速）；
- FA3（2024）：针对 H100（Hopper）新硬件能力，利用 Tensor Core + TMA 的异步性做 warp specialization、块级 matmul 与 softmax 交错，并支持 FP8 低精度 的 incoherent processing；在 H100 上 FP16/BF16 再获 1.5–2.0× 速度提升，FP8 达到接近 1.2 PFLOPs/s，且数值误差优于基线 FP8 注意力；

PyTorch 提供了统一的 scaled_dot_product_attention（SDPA） 接口，会在 CUDA 上自动选择最优后端（包含 FlashAttention-2 后端、内存高效后端等）。你只要调用 SDPA / 或启用相应后端即可享受加速（取决于形状、精度、硬件与构建选项）。

In [3]:
import torch
from torch.nn.functional import scaled_dot_product_attention as sdpa

# (batch, heads, seq, head_dim)
q = torch.randn(2, 16, 2048, 128, device="mps", dtype=torch.bfloat16)
k = torch.randn_like(q)
v = torch.randn_like(q)

# so long as the hardware/dtype/shape meets the conditions, SDPA will automatically select the efficient kernel
out = sdpa(q, k, v, is_causal=True)  # training / inference
out.shape

torch.Size([2, 16, 2048, 128])

常见问题。
- 不是近似注意力：复杂度仍是 O(n²)（算子本身没变），加速来自更少的显存 IO 与更高的并行效率，因此对极长的 n 仍需配合稀疏 / 滑窗等策略时再看场景权衡；
- 形状 / 精度限制：不同实现对 head_dim、dtype、mask 形式等有条件；PyTorch SDPA 会自动回退到其它实现，因此性能可能随输入改变；
- 硬件差异：FA3 的大幅提升是 Hopper 特性驱动的；当然，在 A100 上 FA2 已经很高效，但 PFLOPs 级指标需要 H100 + FP8 支持；